In [22]:
import os
import urllib
import numpy as np
import tensorflow as tf

In [23]:
# Data sets
IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

In [28]:
def main():
    # download data
    if not os.path.exists(IRIS_TRAINING):
        with urllib.request.urlopen(IRIS_TRAINING_URL) as url:
            raw = url.read()
        with open(IRIS_TRAINING, "wb") as f:
            f.write(raw)

    if not os.path.exists(IRIS_TEST):
        with urllib.request.urlopen(IRIS_TEST_URL) as url:
            raw = url.read()        
        with open(IRIS_TEST, "wb") as f:
            f.write(raw)
    
    # load data
    training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
      filename=IRIS_TRAINING,
      target_dtype=np.int,
      features_dtype=np.float32
    )
    test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
      filename=IRIS_TEST,
      target_dtype=np.int,
      features_dtype=np.float32
    )
    
    # specify that all features have real-value data
    feature_columns = [tf.feature_column.numeric_column("x", shape=[4])]
    
    # build 3 layer DNN with 10,20,10 units respectively
    classifier = tf.estimator.DNNClassifier(
        feature_columns=feature_columns,
        hidden_units=[10,20,10,10],
        n_classes=3,
        model_dir="/tmp/iris_model"
    )
    
    # Define the training inputs
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": np.array(training_set.data)},
      y=np.array(training_set.target),
      num_epochs=None,
      shuffle=True
    )
    
    # train model
    classifier.train(input_fn=train_input_fn, steps=2000)
    
    # define test inputs
    test_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": np.array(test_set.data)},
      y=np.array(test_set.target),
      num_epochs=1,
      shuffle=False
    )
    
    # Evaluate accuracy
    accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]
    
    print("\nTest Accuracy: {0:f}\n".format(accuracy_score))
    
    # classify 2 new flower samples
    new_samples = np.array(
      [[6.4, 3.2, 4.5, 1.5],
       [5.8, 3.1, 5.0, 1.7]], dtype=np.float32        
    )
    
    predict_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": new_samples},
      num_epochs=1,
      shuffle=False
    )
    
    predictions = list(classifier.predict(input_fn=predict_input_fn))
    predicted_classes = [p["classes"] for p in predictions]
    
    print(
      "New Samples, Class Predictions:    {}\n"
      .format(predicted_classes))

if __name__ == "__main__":
    main()

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_session_config': None, '_model_dir': '/tmp/iris_model', '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_save_checkpoints_secs': 600}
ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Tensor'>):
<tf.Tensor 'report_uninitialized_variables_1/boolean_mask/Gather:0' shape=(?,) dtype=string>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
['File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py", line 170, in _run_module_as_main\n    "__main__", mod_spec)', 'File "/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/runpy.py", line 85, in _run_code\n    exec(code, run_globals)', 'File "/Library/Frameworks/Python.framework/Versions/3.5/lib/py